In [ ]:
import pandas as pd
import numpy as np
import os # DEBUG
from glob import glob
from pandas_profiling import ProfileReport
import yaml
import re


In [ ]:
PATH_ABS_SRC = os.getcwd()
PATH_REL = os.path.dirname(os.getcwd())

In [ ]:
with open("naming.yaml") as stream:
    naming = yaml.safe_load(stream)

In [ ]:
df = pd.read_csv('../data/profiles_revised.csv')

In [ ]:
df.head()

In [ ]:
profile = ProfileReport(df, title='Pandas Profilign Report')
#profile.to_widgets() # does not show anything
profile.to_notebook_iframe()
profile.to_file("pandas_profiling_data_report.html")

In [ ]:
def print_col_values(list, filename):
    with open(r'{}.txt'.format(filename), 'w') as fp:
        for element in list:
            fp.write("{}\n".format(element))

In [ ]:
# Extract Col
zodiacs = df.sign.unique()
print_col_values(list=zodiacs, filename='zodiacs')

ZODIAC_STRING_REPLACMENT = '&rsquo;' # corresponds to " ' "

# Clean
zodiacs = [z for z in zodiacs if str(z) != 'nan'] # remove nan values
zodiacs = [v.replace('&rsquo;', '\'') for v in zodiacs] # replace '

# Check
print_col_values(list=zodiacs, filename='zodiacs-cleaned')



In [ ]:
# copy by value
df_zodiac = df.copy()

# nan's, and spelling
df_zodiac.dropna(inplace=True, subset=['sign']) # remove nan's
df_zodiac.shape # (48890, 19) , same as profiler

# extract only sign
df_zodiac['sign-extracted'] = df_zodiac['sign'].str.split(' ').str[0]

# extract sign modifier
df_zodiac['sign-modifier-extracted'] = df_zodiac['sign'].str.split(' ').str[1:]
df_zodiac['sign-modifier-extracted'] = df_zodiac['sign-modifier-extracted'].apply(lambda y: '' if len(y)==0 else y) # replace empty lists with ''
df_zodiac['sign-modifier-extracted'] = df_zodiac['sign-modifier-extracted'].apply(lambda y: ' '.join(y) if len(y)!=0 else y) # join list of strings together
df_zodiac['sign-modifier-extracted'] = df_zodiac['sign-modifier-extracted'].str.replace(ZODIAC_STRING_REPLACMENT,'\'')  # replace 

# map sign modifier + ordinal classifier
mapper_naming_dict = naming['zodiac_hiearchy'] 
df_zodiac['sign-modifier-extracted-ordinal'] =  df_zodiac['sign-modifier-extracted'].map(mapper_naming_dict).fillna(df_zodiac['sign-modifier-extracted']) # map values from dict according to string
df_zodiac[['sign', 'sign-extracted', 'sign-modifier-extracted', 'sign-modifier-extracted-ordinal']]